# Trading EDA

To install conda and binance packages to this notebook uncomment the code below

In [1]:
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
#%pip install binance-connector==1.13.0

Init binance client

In [2]:
from binance.spot import Spot as Client
import os
import sys
import yaml

# Load 
def load_config():
    """
    Load config from cfg folder respecting the order: defaults, app.yaml, environment vars
    """
    # Defaults
    cfg_dir="./../biml/cfg"
    default_cfg_path = f"{cfg_dir}/app-defaults.yaml"
    with open(default_cfg_path, "r") as appdefaults:
        config = yaml.safe_load(appdefaults)

    # Custom config, should contain custom information,
    cfg_path = f"{cfg_dir}/app.yaml"
    if os.path.exists(cfg_path):
        with open(cfg_path) as app:
            config.update(yaml.safe_load(app))
    else:
        sys.exit(
            f"Config {cfg_path} not found. Please copy cfg/app-defaults.yaml to {cfg_path} "
            f"and update connection info there.")

    # Enviroment variabless
    config.update(os.environ)
    return config


# Create binance client
config=load_config()
key,secret,url=config["biml.connector.key"],config["biml.connector.secret"],config["biml.connector.url"]
client = Client(key=key, secret=secret, base_url=url, timeout=10)


Get account info

In [3]:
account=client.account()
print(account)


{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'brokered': False, 'updateTime': 1660762371581, 'accountType': 'SPOT', 'balances': [{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '0.99900000', 'locked': '0.00000000'}, {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '10009.29733432', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}], 'permissions': ['SPOT']}


Get orders from binance

In [32]:
# Get orders
import pandas as pd
symbol="BTCUSDT"
orders_raw = client.get_orders(symbol)
orders=pd.DataFrame(orders_raw)
if not orders.empty: orders=orders.astype({'time': 'datetime64[ms]','updateTime':'datetime64[ms]'}).set_index('time',drop=False)
orders.tail()

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,origQuoteOrderQty,trailingDelta
time,,,,,,,,,,,,,,,,,,,
2022-08-17 15:53:28.068,BTCUSDT,7123301,-1,eF3CbFcJnZKeWHy48XWMqu,24530.06000000,0.00100000,0.00000000,0.00000000,CANCELED,GTC,STOP_LOSS_LIMIT,BUY,23946.01000000,0.00000000,2022-08-17 15:53:28.068,2022-08-17 18:50:53.404,False,0.00000000,500.0
2022-08-17 18:50:54.205,BTCUSDT,7192538,-1,s5EY9FGyO0KWRsNP08mfl1,0.00000000,0.00100000,0.00100000,23.53503000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-08-17 18:50:54.205,2022-08-17 18:50:54.205,True,0.00000000,NaN
2022-08-17 18:52:43.678,BTCUSDT,7193092,-1,V9OsOn7kQJ5BC4kR2Ihtu6,23532.16000000,0.00100000,0.00100000,23.53216000,FILLED,GTC,LIMIT,BUY,0.00000000,0.00000000,2022-08-17 18:52:43.678,2022-08-17 18:52:51.581,True,0.00000000,NaN
2022-08-17 18:59:31.342,BTCUSDT,7195090,-1,aswDQZtfqV9JfuFKJ7wClh,23451.22000000,0.00100000,0.00100000,23.45122000,FILLED,GTC,LIMIT,BUY,0.00000000,0.00000000,2022-08-17 18:59:31.342,2022-08-17 18:59:49.675,True,0.00000000,NaN
2022-08-17 19:01:51.295,BTCUSDT,7195787,-1,zIq1wB4KO3951FXDP7ugR3,23456.82000000,0.00100000,0.00100000,23.45682000,FILLED,GTC,LIMIT,SELL,0.00000000,0.00000000,2022-08-17 19:01:51.295,2022-08-17 19:02:08.461,True,0.00000000,NaN


Get trades from Binance

In [54]:
orders_opened=client.get_open_orders(symbol)
print(orders_opened)

[]


In [72]:
import pandas as pd
import numpy as np

#mytrades_dict=[{'symbol': 'BTCUSDT', 'id': 882731, 'orderId': 1957295, 'orderListId': 367, 'price': '21900.00000000', 'qty': '0.00100000', 'quoteQty': '21.90000000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'time': 1655261128431, 'isBuyer': False, 'isMaker': True, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 907300, 'orderId': 2003764, 'orderListId': -1, 'price': '20860.34000000', 'qty': '0.00100000', 'quoteQty': '20.86034000', 'commission': '0.00000000', 'commissionAsset': 'BTC', 'time': 1655271037368, 'isBuyer': True, 'isMaker': False, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 2860170, 'orderId': 6104147, 'orderListId': -1, 'price': '21481.31000000', 'qty': '0.00100000', 'quoteQty': '21.48131000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'time': 1656149861641, 'isBuyer': False, 'isMaker': True, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 2901517, 'orderId': 6214259, 'orderListId': -1, 'price': '20981.74000000', 'qty': '0.00100000', 'quoteQty': '20.98174000', 'commission': '0.00000000', 'commissionAsset': 'BTC', 'time': 1656172361645, 'isBuyer': True, 'isMaker': False, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 2901645, 'orderId': 6214506, 'orderListId': -1, 'price': '21000.00000000', 'qty': '0.00100000', 'quoteQty': '21.00000000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'time': 1656172400009, 'isBuyer': False, 'isMaker': False, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 2901714, 'orderId': 6214673, 'orderListId': -1, 'price': '20970.56000000', 'qty': '0.00100000', 'quoteQty': '20.97056000', 'commission': '0.00000000', 'commissionAsset': 'BTC', 'time': 1656172432877, 'isBuyer': True, 'isMaker': False, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 2987743, 'orderId': 6486737, 'orderListId': -1, 'price': '21373.90000000', 'qty': '0.00100000', 'quoteQty': '21.37390000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'time': 1656229949510, 'isBuyer': False, 'isMaker': True, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 2987760, 'orderId': 6486675, 'orderListId': -1, 'price': '21377.89000000', 'qty': '0.00100000', 'quoteQty': '21.37789000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'time': 1656229952525, 'isBuyer': False, 'isMaker': True, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 3063166, 'orderId': 6717004, 'orderListId': -1, 'price': '21336.61000000', 'qty': '0.00200000', 'quoteQty': '42.67322000', 'commission': '0.00000000', 'commissionAsset': 'BTC', 'time': 1656278728249, 'isBuyer': True, 'isMaker': False, 'isBestMatch': True}]
symbol="BTCUSDT"
mytrades_dict = client.my_trades(symbol)

# Exclude first order if it was closing order
if orders_opened:
    # Last order is opened, so total amount is odd
    if len(mytrades_dict)%2 == 0:
        mytrades_dict = mytrades_dict[1:]
elif len(mytrades_dict)%2 == 1:
    # Last oder is closed, so total amount is even
    mytrades_dict = mytrades_dict[1:]

# Preprocess trades df
mytrades=pd.DataFrame(mytrades_dict)
mytrades=mytrades.astype({'time': 'datetime64[ms]'}).set_index('time',drop=False)
mytrades.qty=np.where(mytrades.isBuyer==False,mytrades.qty.astype('float')*-1,mytrades.qty.astype('float'))
mytrades.loc[mytrades.index[0], "qty"] = 0
mytrades["price"]=mytrades["price"].astype('float64')
mytrades["qty"]=mytrades["qty"].astype('float64')
mytrades["qtycumsum"]=mytrades["qty"].cumsum()

mytrades.tail()

,symbol,id,orderId,orderListId,price,qty,quoteQty,commission,commissionAsset,time,isBuyer,isMaker,isBestMatch,qtycumsum
time,,,,,,,,,,,,,,
2022-08-17 15:53:33.139,BTCUSDT,2137182,7123295,-1,23361.96,-0.001,23.36196000,0.00000000,USDT,2022-08-17 15:53:33.139,False,True,True,-0.002
2022-08-17 18:50:54.205,BTCUSDT,2157712,7192538,-1,23535.03,0.001,23.53503000,0.00000000,BTC,2022-08-17 18:50:54.205,True,False,True,-0.001
2022-08-17 18:52:51.581,BTCUSDT,2157930,7193092,-1,23532.16,0.001,23.53216000,0.00000000,BTC,2022-08-17 18:52:51.581,True,True,True,0.000
2022-08-17 18:59:49.675,BTCUSDT,2158665,7195090,-1,23451.22,0.001,23.45122000,0.00000000,BTC,2022-08-17 18:59:49.675,True,True,True,0.001
2022-08-17 19:02:08.461,BTCUSDT,2158952,7195787,-1,23456.82,-0.001,23.45682000,0.00000000,USDT,2022-08-17 19:02:08.461,False,True,True,0.000


Get candles from local history starting from earliest trade date ending by latest trade date.

In [70]:
import os
import glob
import pandas as pd
from datetime import timedelta,date


def read_candles(start_date_inc, end_date_inc):
    """ Read days from start to end from local history"""
    symbol="BTCUSDT"    
    data_dir=f"./../data/{symbol}"
    print(f"start date:{start_date_inc}, end date:{end_date_inc}, all inclusive")
    candles = pd.DataFrame()
    cur_candles_list=[]
    for n in range(int ((end_date_inc - start_date_inc).days)+1):
        cur_date = start_date_inc + timedelta(n)    
        csv_path=f"{data_dir}/{cur_date}_{symbol}_1m.csv"     
        if not os.path.exists(csv_path): continue
        #print(f"Read {cur_date} candles from {csv_path}")
        cur_candles=pd.read_csv(csv_path,index_col="close_time")[["open","high","low","close","vol"]]
        cur_candles_list.append(cur_candles)
    candles = pd.concat(cur_candles_list)
    return candles

def clean_candles(df):
    """ Remove outliers """
    q_low = df["low"].quantile(0.01)
    q_high  = df["high"].quantile(0.99)
    df=df[df["high"]< q_high]
    df=df[df["low"] > q_low ]
    return df

# Read candles for last 3 days
last_date=mytrades.index.max().date()
first_date=last_date - timedelta(days=0)

candles = read_candles(first_date, last_date)
candles = clean_candles(candles)
candles.tail()


start date:2022-08-17, end date:2022-08-17, all inclusive


,open,high,low,close,vol
close_time,,,,,
2022-08-17 19:39:59.999,23300.75,23300.75,23294.98,23298.89,3.326903
2022-08-17 19:40:59.999,23296.80,23306.79,23296.80,23298.75,4.910206
2022-08-17 19:41:59.999,23298.75,23314.43,23298.75,23307.60,5.002167
2022-08-17 19:42:59.999,23310.29,23326.11,23306.60,23306.60,5.882370
2022-08-17 19:43:59.999,23306.60,23316.40,23302.96,23308.65,3.529390


## EDA

### Look at trades quantity cumulative sum

In [73]:
df=mytrades.copy()[["symbol","price","qty","qtycumsum"]]
df.price=df.price.astype("float")
df.tail()

,symbol,price,qty,qtycumsum
time,,,,
2022-08-17 15:53:33.139,BTCUSDT,23361.96,-0.001,-0.002
2022-08-17 18:50:54.205,BTCUSDT,23535.03,0.001,-0.001
2022-08-17 18:52:51.581,BTCUSDT,23532.16,0.001,0.000
2022-08-17 18:59:49.675,BTCUSDT,23451.22,0.001,0.001
2022-08-17 19:02:08.461,BTCUSDT,23456.82,-0.001,0.000


## Look at trades on candlestick chart

In [74]:
# Install a conda package in the current Jupyter kernel
#%conda install -c plotly plotly=5.9.0
# Draw candlechart
import plotly as py
from plotly import graph_objects as go
import pandas as pd

df=candles.copy()
#df=clean_candles(df)
fig = go.Figure(data=[ \
                    go.Candlestick(\
                        x=df.index,\
                        open=df['open'],\
                        high=df['high'],\
                        low=df['low'],\
                        close=df['close'])
                     ])

# Customize ay to distribute captions by y so they do not overlap by x
ay,aymax,aystep=0,50,10
# Annotate buy, sell on the chart
for traderec in mytrades[mytrades.index>df.index.min()].iterrows():
    ay=(ay+aystep)%aymax
    trade=traderec[1]
    color='green' if trade['isBuyer'] else 'red'
    text_short='buy' if trade['isBuyer'] else 'sell'
    text=f"{trade['time']} {text_short} at {trade['price']}, cumsum: {str(trade['qtycumsum'])}"
    fig.add_annotation(x=trade['time'], \
                       y=trade['price'],\
                       text=text_short,\
                       hovertext=text,\
                       showarrow=True,arrowhead=5,arrowcolor=color,ay=-ay,arrowwidth=2,
                       font={'color':color}
                      )
fig.show()

## Trades

In [76]:
import matplotlib.pyplot as plt
import plotly.express as px
# Calc profit, considering only closing trades
profit=mytrades[["qty","price"]].copy()
profit["profit"]=profit["price"].diff()*(profit["qty"]/profit["qty"])
profit = profit.iloc[1::2,:]
profit["cum_profit"] = profit["profit"].cumsum()

#
fig = px.line(profit, y="cum_profit")
fig.show()
#profit["profit"].plot()
#plt.show()


## Look at account

In [77]:
from datetime import datetime as dt
myms=1660277364145
print(dt.fromtimestamp(myms / 1000))


#orders.tail(20)
orders[(orders["type"]=="MARKET") & (orders["side"]=="BUY")].tail(20)

2022-08-12 07:09:24.145000


,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,origQuoteOrderQty,trailingDelta
time,,,,,,,,,,,,,,,,,,,
2022-08-16 15:04:45.328,BTCUSDT,6524450,-1,5bvtR9zvkLLcIHNUxI90DL,0.00000000,0.00100000,0.00100000,23.70729000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-08-16 15:04:45.328,2022-08-16 15:04:45.328,True,0.00000000,NaN
2022-08-16 15:06:57.702,BTCUSDT,6525351,-1,MJMnbJ4IO5ox8aO6CnBqXF,0.00000000,0.00100000,0.00100000,23.70817000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-08-16 15:06:57.702,2022-08-16 15:06:57.702,True,0.00000000,NaN
2022-08-16 15:09:10.438,BTCUSDT,6526247,-1,VOszTfXDPdPUqd4XgWEuO2,0.00000000,0.00100000,0.00100000,23.73510000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-08-16 15:09:10.438,2022-08-16 15:09:10.438,True,0.00000000,NaN
2022-08-16 15:11:39.745,BTCUSDT,6527242,-1,zG7ZJUfOw0v5mYhu5uvYbI,0.00000000,0.00100000,0.00100000,23.71397000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-08-16 15:11:39.745,2022-08-16 15:11:39.745,True,0.00000000,NaN
2022-08-16 15:11:53.039,BTCUSDT,6527333,-1,eANlNNyD2shyOjw0wSJB2U,0.00000000,0.00100000,0.00100000,23.71464000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-08-16 15:11:53.039,2022-08-16 15:11:53.039,True,0.00000000,NaN
2022-08-16 15:12:09.468,BTCUSDT,6527455,-1,8YM18Wyz1fwJhm7Ac4DQCJ,0.00000000,0.00100000,0.00100000,23.72888000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-08-16 15:12:09.468,2022-08-16 15:12:09.468,True,0.00000000,NaN
2022-08-16 15:14:03.419,BTCUSDT,6528284,-1,D2mTrJrKgKXv3wVJ5PqMKB,0.00000000,0.00100000,0.00100000,23.77219000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-08-16 15:14:03.419,2022-08-16 15:14:03.419,True,0.00000000,NaN
2022-08-16 15:16:30.871,BTCUSDT,6529325,-1,hc24GlhETcySAXgx3REDz4,0.00000000,0.00100000,0.00100000,23.76367000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-08-16 15:16:30.871,2022-08-16 15:16:30.871,True,0.00000000,NaN
2022-08-16 15:16:44.668,BTCUSDT,6529419,-1,YP5WOEKAvHDqscWI7LJKTm,0.00000000,0.00100000,0.00100000,23.77287000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-08-16 15:16:44.668,2022-08-16 15:16:44.668,True,0.00000000,NaN


In [14]:
print(account)

{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'brokered': False, 'updateTime': 1660762371581, 'accountType': 'SPOT', 'balances': [{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '0.99900000', 'locked': '0.00000000'}, {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '10009.29733432', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}], 'permissions': ['SPOT']}
